<a href="https://colab.research.google.com/github/ownit4137/TIL/blob/main/DL%20from%20Scratch/1/BatchNormalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 배치 정규화

$ \hat{x_i}  \leftarrow   \cfrac{x_i - \mu_B}{\sqrt{{\sigma_B^2} + \epsilon}} $

$y_i = \gamma \hat{x_i} + \beta$

- [논문](https://arxiv.org/abs/1502.03167)
- 속도 향상, 초깃값 선택 문제 해결, 과적합 해결등 많은 장점을 가짐
- 활성화 함수의 앞, 뒤에서 미니배치 단위의 정규화(평균 0, 분산 1)가 일어남
- 정규화 후 확대와 이동 변환 수행
- Internal Convarient Shift 문제를 해결
- [ref](https://eehoeskrap.tistory.com/430)
- [ref](https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/)
- [ref](https://gaussian37.github.io/dl-concept-batchnorm/)


In [ ]:
class BatchNormalization:
   
  def __forward(self, x, train_flg):
    ...

      # 정규화
      mu = x.mean(axis=0)
      xc = x - mu
      var = np.mean(xc**2, axis=0)
      std = np.sqrt(var + 10e-7)
      xn = xc / std

    ... 
        
    out = self.gamma * xn + self.beta 
    return out

  def __backward(self, dout):

    # 역전파로 각 변수 갱신
    dbeta = dout.sum(axis=0)
    dgamma = np.sum(self.xn * dout, axis=0)

    dxn = self.gamma * dout
    dxc = dxn / self.std
    dstd = -np.sum((dxn * self.xc) / (self.std * self.std), axis=0)
    dvar = 0.5 * dstd / self.std
    dxc += (2.0 / self.batch_size) * self.xc * dvar
    dmu = np.sum(dxc, axis=0)
    dx = dxc - dmu / self.batch_size
    
    self.dgamma = dgamma
    self.dbeta = dbeta
    
    return dx